<a href="https://colab.research.google.com/github/iqbalrez/modified-GMOCAT-colab/blob/main/GMOCAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Initial**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.1/355.1 MB 4.0 MB/s eta 0:00:00


In [ ]:
pip install vegas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.5 MB/s eta 0:00:00


# **Data Preprocessing**

In [ ]:
import os
import numpy as np
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from multiprocessing import Pool
import argparse
from collections import defaultdict

question_map = {}
question_meta, subject_metadata, df, question_meta_1, question_meta_3, = {}, {}, {}, {}, {}

date_format = "%Y-%m-%d %H:%M:%S.%f %z"

def open_json(path_):
    with open(path_) as fh:
        data = json.load(fh)
    return data


def dump_json(path_, data):
    with open(path_, 'w') as fh:
        json.dump(data, fh, indent=2)
    return data

def f_dbekt22(uuid):
    global df, question_map
    user_df = df[df.student_id == uuid].sort_values('student_id')
    q_ids, labels, times = [], [], []
    q_ids_set = set()
    for _, row in user_df.iterrows():
        q_id = str(row['question_id'])
        if q_id in q_ids_set:
            continue
        q_ids_set.add(q_id)
        q_ids.append(question_map[q_id])
        ans = 1 if row['answer_state'] else 0
        labels.append(ans)
    out = {'student_id': int(uuid), 'q_ids': q_ids, 'labels': labels, 'log_num':len(labels)}
    return out

def featurize_dbekt22():
    global question_map, df
    df = pd.read_csv('raw_data/DBEKT22/datasets/Transaction.csv', encoding = 'ISO-8859-1', low_memory=False,
                     usecols=['student_id', 'question_id', 'answer_state']).dropna()

    q2k_df = pd.read_csv('raw_data/DBEKT22/datasets/Question_KC_Relationships.csv', encoding = 'ISO-8859-1', low_memory=False).dropna()

    print(df.dtypes)
    print(q2k_df.dtypes)

    user_ids = df['student_id'].unique()
    problems = df['question_id'].unique()

    for p in problems:
        question_map[str(p)] = len(question_map)
    with Pool(30) as p:
        results = p.map(f_dbekt22, user_ids)

    bad_interactions = [len(d['q_ids']) for d in results if len(d['q_ids']) < 40]
    results = [d for d in results if len(d['q_ids']) >= 40]
    interactions = [len(d['q_ids']) for d in results]

    table = q2k_df.loc[:, ['question_id', 'knowledgecomponent_id']].drop_duplicates()
    q2k = {}
    k2n = {}
    for _,row in table.iterrows():
        qid = str(question_map[str(row['question_id'])])
        kid = str([row['knowledgecomponent_id']])
        if kid not in k2n:
            k2n[kid] = len(k2n)

        if qid not in q2k:
          q2k[qid] = []  # Inisialisasi daftar untuk qid jika belum ada

        if k2n[kid] not in q2k[qid]:
          q2k[qid].append(k2n[kid]) # Tambah data KC

    print('Number of DBEKT22 User: ', len(results))
    print('Number of DBEKT22 Interactions: ', sum(interactions))
    print('Ignored DBEKT22 Interactions: ', sum(bad_interactions))
    print('Number of Problems DBEKT22:', len(question_map))
    print('Number of Knowledge DBEKT22:', len(k2n))

    dump_json('data/train_task_dbekt22.json', results)
    dump_json('data/question_map_dbekt22.json', question_map)
    dump_json('data/concept_map_dbekt22.json', q2k)

featurize_dbekt22()

answer_state     bool
student_id      int64
question_id     int64
dtype: object
id                       int64
question_id              int64
knowledgecomponent_id    int64
dtype: object
Number of DBEKT22 User:  991
Number of DBEKT22 Interactions:  155309
Ignored DBEKT22 Interactions:  3707
Number of Problems DBEKT22: 212
Number of Knowledge DBEKT22: 93


In [ ]:
import json
import pandas as pd

def open_json(path_):
    with open(path_) as fh:
        data = json.load(fh)
    return data

def dump_json(path_, data):
    with open(path_, 'w') as fh:
        json.dump(data, fh, indent=2)
    return data

def map_question_text():
    question_map = open_json('data/question_map_dbekt22')

    question_df = pd.read_csv('raw_data/DBEKT22/datasets/Questions.csv', encoding='ISO-8859-1', low_memory=False,
                     usecols=['id', 'question_text']).dropna()
    choices_df = pd.read_csv('raw_data/DBEKT22/datasets/Question_Choices.csv', encoding='ISO-8859-1', low_memory=False, usecols =['question_id','choice_text','is_correct']).dropna()
    question_text_map = {}
    for _, row in question_df.iterrows():
        original_question_id = str(row['id'])
        question_text = row['question_text']
        if original_question_id in question_map:
            mapped_question_id = question_map[original_question_id]

            # Filter choices
            choices = choices_df[choices_df['question_id'] == int(original_question_id)]
            # Format choices and find the correct answer
            formatted_choices = []
            correct_answer = None
            for _, choice_row in choices.iterrows():
                choice_text = choice_row['choice_text']
                is_correct = choice_row['is_correct']
                formatted_choices.append(choice_text)
                if is_correct:  # Check if the choice is the correct answer
                    correct_answer = choice_text

            # Add question with choices and correct answer to the map
            question_text_map[mapped_question_id] = {
                'question_text': question_text,
                'choices': formatted_choices,
                'correct_answer': correct_answer
            }

    # Simpan ke file JSON
    dump_json('data/question_text_map_dbekt22.json', question_text_map)
    print('Number of Problems DBEKT22:', len(question_text_map))

# Jalankan fungsi untuk membuat JSON
map_question_text()

Number of Problems DBEKT22: 212


# **Construct Graph & Novelty Reward**

In [ ]:
import json
import numpy as np
from collections import Counter
import json

def dump_json(path_, data):
    with open(path_, 'w') as fh:
        json.dump(data, fh, indent=2)
    return data

def build_local_map(name):
    concept_file = f'data/concept_map_{name}.json'
    if name == 'assist2009':
        exer_n = 17751+1
    if name == 'junyi':
        exer_n = 2835+1
    if name == '3_4':
        exer_n = 948+1
    if name == 'dbekt22':
        exer_n = 212+1

    temp_list = []
    with open(concept_file, encoding='utf8') as f:
        concept_map = json.load(f)
    k_from_e = '' # e(src) to k(dst)
    e_from_k = '' # k(src) to k(dst)

    for qid in concept_map:
        # has id=0 question for pad
        exer_id = int(qid) + 1
        for k in concept_map[str(qid)]:
            if (str(exer_id) + '\t' + str(k + exer_n)) not in temp_list or (str(k + exer_n) + '\t' + str(exer_id)) not in temp_list:
                k_from_e += str(exer_id) + '\t' + str(k + exer_n) + '\n'
                e_from_k += str(k + exer_n) + '\t' + str(exer_id) + '\n'
                temp_list.append((str(exer_id) + '\t' + str(k + exer_n)))
                temp_list.append((str(k + exer_n) + '\t' + str(exer_id)))

    path = f'graph_data/{name}/'

    with open(path+'k_from_e.txt', 'w') as f:
        f.write(k_from_e)
    with open(path+'e_from_k.txt', 'w') as f:
        f.write(e_from_k)

def constructDependencyMatrix(name):
    data_file = f'data/train_task_{name}.json'
    concept_file = f'data/concept_map_{name}.json'
    if name == 'assist2009':
        knowledge_n = 123 # num of knowledge
    if name == 'junyi':
        knowledge_n = 40
    if name == '3_4':
        knowledge_n = 86
    if name == 'dbekt22':
        knowledge_n = 93+1

    edge_dic_deno = {}
    with open(data_file, encoding='utf8') as i_f:
        data = json.load(i_f)

    with open(concept_file, encoding='utf8') as f:
        concept_map = json.load(f)
    # Calculate correct matrix
    knowledgeCorrect = np.zeros([knowledge_n, knowledge_n])
    for student in data:
        if student['log_num'] < 2:
            continue
        q_ids, labels = student['q_ids'], student['labels']
        for log_i in range(student['log_num']-1):
            if labels[log_i] * labels[log_i+1] == 1:
                for ki in concept_map[str(q_ids[log_i])]:
                    for kj in concept_map[str(q_ids[log_i+1])]:
                        if ki != kj:
                            # n_{ij}
                            knowledgeCorrect[ki][kj] += 1.0
                            # n_{i*}, calculate the number of correctly answering i
                            if ki in edge_dic_deno.keys():
                                edge_dic_deno[ki] += 1
                            else:
                                edge_dic_deno[ki] = 1

    s = 0
    c = 0
    # Calculate transition matrix
    knowledgeDirected = np.zeros([knowledge_n, knowledge_n])
    for i in range(knowledge_n):
        for j in range(knowledge_n):
            if i != j and knowledgeCorrect[i][j] > 0:
                    knowledgeDirected[i][j] = float(knowledgeCorrect[i][j]) / edge_dic_deno[i]
                    s += knowledgeDirected[i][j]
                    c += 1
    o = np.zeros([knowledge_n, knowledge_n])
    min_c = 100000000
    max_c = 0
    for i in range(knowledge_n):
        for j in range(knowledge_n):
            if knowledgeCorrect[i][j] > 0 and i != j:
                min_c = min(min_c, knowledgeDirected[i][j])
                max_c = max(max_c, knowledgeDirected[i][j])
    s_o = 0
    l_o = 0
    for i in range(knowledge_n):
        for j in range(knowledge_n):
            if knowledgeCorrect[i][j] > 0 and i != j:
                o[i][j] = (knowledgeDirected[i][j] - min_c) / (max_c - min_c)
                l_o += 1
                s_o += o[i][j]

    # avg = 0.02
    if name == 'assist2009':
        avg = s_o / l_o #total / count
        avg *= avg
        avg *= avg
    elif name == '3_4':
        avg = s_o / l_o #total / count # 0.02
        # avg =0.02
    else:
        avg = s_o / l_o #total / count
        # avg *= avg
        # avg *= avg

    print(avg)
    # avg is threshold
    graph = ''
    # edge = np.zeros([knowledge_n, knowledge_n])
    for i in range(knowledge_n):
        for j in range(knowledge_n):
            if o[i][j] >= avg:
                graph += str(i) + '\t' + str(j) + '\n'
                # edge[i][j] = 1
    path = f'graph_data/{name}/'
    with open(path+'knowledgeGraph.txt', 'w') as f:
        f.write(graph)

def process_edge(name):
    K_Directed = ''
    K_Undirected = ''
    edge = []
    path = f'graph_data/{name}/'
    with open(path+'knowledgeGraph.txt', 'r') as f:
        for i in f.readlines():
            i = i.replace('\n', '').split('\t')
            src = i[0]
            tar = i[1]
            edge.append((src, tar))
    visit = []
    for e in edge:
        if e not in visit:
            if (e[1],e[0]) in edge:
                K_Undirected += str(e[0] + '\t' + e[1] + '\n')
                visit.append(e)
                visit.append((e[1],e[0]))
            else:
                K_Directed += str(e[0] + '\t' + e[1] + '\n')
                visit.append(e)

    with open(path+'K_Directed.txt', 'w') as f:
        f.write(K_Directed)
    with open(path+'K_Undirected.txt', 'w') as f:
        f.write(K_Undirected)
    all = len(visit)
    print(all)

def nov_reward(dataset):
    data_file = f'data/train_task_{dataset}.json'
    with open(data_file, encoding='utf8') as i_f:
        data = json.load(i_f)

    all_questions = []
    for student in data:
        q_ids = student['q_ids']
        all_questions.extend(q_ids)
    print(min(all_questions))
    # Get Novel Items
    all_pairs = Counter(all_questions).items()
    item_freqs = [pair[1] for pair in all_pairs]

    threshold = np.quantile(item_freqs, q=0.9)
    print(threshold)

    less_popular_items = []
    for pair in all_pairs:
        if pair[1] <= threshold:
            less_popular_items.append(pair[0])
    print('number of less popular items is: ', len(less_popular_items))

    # Binary Novelty Reward System
    binary_nov_reward= {}
    for pair in all_pairs:
        if pair[1] <= threshold:
            binary_nov_reward[str(pair[0])] = 1
        else:
            binary_nov_reward[str(pair[0])] = 0

    dump_json(f'data/nov_reward_{dataset}.json', binary_nov_reward)


name = 'dbekt22'
build_local_map(name)
constructDependencyMatrix(name)
process_edge(name)
nov_reward(name)

0.028564666972944908
272
0
824.0
number of less popular items is:  191


# **NCDM Pretrain**

In [ ]:
!bash python ./pretrain.py -T 20 -seed 145 \
-learning_rate 0.002 -training_epoch 100 -batch 2048 \
-CDM NCD -gpu_no 0 -data_name dbekt22

user num 991 991
use user num:  793
Namespace(seed=145, data_name='dbekt22', CDM='NCD', T=20, gpu_no='0', learning_rate=0.002, training_epoch=100, batch_size=2048, path='NCD_dbekt22_20')


In [ ]:
!bash python /content/drive/MyDrive/Skripsi/GMOCAT-modif/launch_adaptive_test.py -environment GCATEnv -T 20 -ST [1,5,10,20,25,30,40,50] -agent GCATAgent -FA GCAT -seed 145 -emb_dim 128 -cdm_bs 128 \
-training_epoch 50 -train_bs 128 -test_bs 1024 -learning_rate 0.001 -policy_epoch 1 \
-gamma 0.5 -n_head 1 -n_block 1 -dropout_rate 0.1 -graph_block 1 \
-morl_weights [1,1,1] -CDM NCD -data_name dbekt22 -gpu_no 0 -cdm_lr 0.02 -cdm_epoch 4 \
-student_ids [54]

DGL backend not selected or invalid.  Assuming PyTorch for now.
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
=== Tes dimulai ===

Step 1:
Coverage: 0.0
Reward: [0.0, 0.0, 0.0]
Apakah Anda siap? 
A.Ya
Masukkan jawaban Anda: A

error[22]: 0.019827336072921753

error[14]: 0.019736647605895996

error[16]: 0.01975882053375244

error[17]: 0.019794106483459473

Step 2:
Coverage: 0.05063291139240506
Reward: [[0.05405405 1.         1.        ]]
Question ID: 144
How many rows will be in the result for the following relational algebra expression?
  A. 6
  B. 5
  C. 4
  D. 3
Masukkan jawaban Anda (A, B, C, atau D): A
Salah.

error[29]: 0.016293227672576904

error[23]: 0.016602158546447754

Step 3:
Coverage: 0.0759493670886076
Reward: [[-0.02702703  1.          1.        ]]
Question ID: 32
In which of the following set definitions is A - B equal

# **GMOCAT Master - DBEKT22**

In [ ]:
basedir = '/content/drive/MyDrive/Skripsi/' # sesuaikan

%cd $basedir/GMOCAT-master

##Train-test

In [ ]:
!bash python ./launch_gcat.py -environment GCATEnv -T 20 -ST [1,5,10,20] -agent GCATAgent -FA GCAT -seed 145 -emb_dim 128 -cdm_bs 128 \
-training_epoch 50 -train_bs 128 -test_bs 1024 -learning_rate 0.001 -policy_epoch 1 \
-gamma 0.5 -n_head 1 -n_block 1 -dropout_rate 0.1 -graph_block 1 \
-morl_weights [1,1,1] -CDM NCD -data_name dbekt22 -gpu_no 0 -cdm_lr 0.02 -cdm_epoch 4

##Run simulation

In [ ]:
!bash python ./launch_adaptive_test.py -environment GCATEnv -T 20 -ST [1,5,10,20,25,30,40,50] -agent GCATAgent -FA GCAT -seed 145 -emb_dim 128 -cdm_bs 128 \
-training_epoch 50 -train_bs 128 -test_bs 1024 -learning_rate 0.001 -policy_epoch 1 \
-gamma 0.5 -n_head 1 -n_block 1 -dropout_rate 0.1 -graph_block 1 \
-morl_weights [1,1,1] -CDM NCD -data_name dbekt22 -gpu_no 0 -cdm_lr 0.02 -cdm_epoch 4 \
-student_ids [102]

=== Tes dimulai ===

Step 1:
Question ID: 0
In the relational data model, a table is considered as____________.
  A. Range
  B. Domain
  C. Relation
  D. Tuple
Masukkan jawaban Anda (A, B, C, atau D): b
Salah.


Coverage: 0.016666666666666666
Reward: [[0.0625 1.     1.    ]]

Step 2:
Question ID: 145
How many rows will be in the result for the following relational algebra expression?
  A. 4
  B. 3
  C. 2
  D. 1
Masukkan jawaban Anda (A, B, C, atau D): a
Salah.


Coverage: 0.03333333333333333
Reward: [[-0.0625  1.      1.    ]]

Step 3:
Question ID: 156
Consider the following database schema: Movie(title, production year, country, run time, major genre) primary key : {title, production year} Person(id, first name, last name, year born) primary key : {id} Director(id, title, production year) primary key : {title, production year} foreign keys : [title, production year] â Movie[title, production year], [id] â Person[id] Writer(id, title, production year, credits) primary key : {id, ti

# **GMOCAT Modif - DBEKT22**

In [ ]:
basedir = '/content/drive/MyDrive/Skripsi/'

%cd $basedir/GMOCAT-modif

/content/drive/MyDrive/Skripsi/GMOCAT-modif


##Train-test

In [ ]:
!bash python ./launch_gcat.py -environment GCATEnv -T 20 -ST [1,5,10,20] -agent GCATAgent -FA GCAT -seed 145 -emb_dim 128 -cdm_bs 128 \
-training_epoch 50 -train_bs 128 -test_bs 1024 -learning_rate 0.001 -policy_epoch 1 \
-gamma 0.5 -n_head 1 -n_block 1 -dropout_rate 0.1 -graph_block 1 \
-morl_weights [1,1,1] -CDM NCD -data_name dbekt22 -gpu_no 0 -cdm_lr 0.02 -cdm_epoch 4

##Run Simulation

In [ ]:
!bash python ./launch_adaptive_test.py -environment GCATEnv -T 20 -ST [1,5,10,20,25,30,40,50] -agent GCATAgent -FA GCAT -seed 145 -emb_dim 128 -cdm_bs 128 \
-training_epoch 50 -train_bs 128 -test_bs 1024 -learning_rate 0.001 -policy_epoch 1 \
-gamma 0.5 -n_head 1 -n_block 1 -dropout_rate 0.1 -graph_block 1 \
-morl_weights [1,1,1] -CDM NCD -data_name dbekt22 -gpu_no 0 -cdm_lr 0.02 -cdm_epoch 4 \
-student_ids [102]